# Textcuboid


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import re
import string
import json
import keras
import wordninja
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

In [3]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

In [4]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [5]:
X=X['title']+' '+X['text']

X = X.apply(clean_text)

In [6]:
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

In [7]:
clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

In [8]:
fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

In [9]:
fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

In [10]:
real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)

In [11]:
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
to_txt=x_train+x_test

In [14]:
def split_compound_words(text):
    return ' '.join(wordninja.split(text))

to_txt_split=[]
for i in to_txt:
    to_txt_split.append(split_compound_words(i))

to_txt=to_txt_split

In [15]:
#문서 길이를 조정하는 코드 텍스트 리스트와 최대 길이를 입력하면 단어 시퀀스를 최대 길이 이내로 truncate 해줌
def limit_words(text_list, max_words):
    new_list = []
    for text in text_list:
        words = text.split()
        if len(words) > max_words:
            words = words[:max_words]
        new_text = ' '.join(words)
        new_list.append(new_text)
    return new_list

In [16]:
#사전학습된 버트는 최대 임베딩 토큰 수(512)가 정해져 있으므로 넉넉하게 최대길이를 300으로 제한
to_txt=limit_words(to_txt,300)

In [17]:
#수동으로 cross validation을 하기위한 코드
cnt=5  #cnt가 2일때 두번째 시행
cnt=cnt-2

#텍스트 큐보이드 생성을 위한 인덱스
def train_idx_list(cnt):
    original_train_lst=[i for i in range(16000)]
    return original_train_lst[:cnt*4000]+original_train_lst[cnt*4000+4000:16000]

def test_idx_list(cnt):
    original_train_lst=[i for i in range(16000)]
    return original_train_lst[cnt*4000:cnt*4000+4000]
    
if cnt>-1:
    train_idx=train_idx_list(cnt)
    test_idx=test_idx_list(cnt)

In [18]:
#불용어 불러오기
with open('C:/Users/user/Desktop/english.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [19]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names())
sw=word_view[TorF]

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
#선별된 10000개 단어
list(sw)

['aaaa',
 'aaf',
 'aap',
 'aardvark',
 'ab',
 'abandons',
 'abated',
 'abba',
 'abbas',
 'abbasi',
 'abbott',
 'abbottabad',
 'abd',
 'abdullah',
 'abel',
 'abell',
 'abella',
 'abercrombie',
 'aberdeen',
 'aberration',
 'abhor',
 'abhorrent',
 'abid',
 'abidin',
 'ablaze',
 'abolished',
 'aboriginal',
 'abortion',
 'abortive',
 'abou',
 'abra',
 'abraham',
 'absences',
 'absentia',
 'absolute',
 'absorb',
 'abstaining',
 'abstentions',
 'absurd',
 'absurdity',
 'abu',
 'abuja',
 'abundance',
 'abuse',
 'abuses',
 'academia',
 'academics',
 'academies',
 'acadia',
 'accelerate',
 'accident',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accommodating',
 'accommodations',
 'accompaniment',
 'accomplishment',
 'accomplishments',
 'accord',
 'account',
 'accumulate',
 'accumulating',
 'accurate',
 'accurately',
 'accusations',
 'accuser',
 'accusers',
 'accuses',
 'acerbic',
 'acetate',
 'acheron',
 'achi',
 'achieve',
 'achilles',
 'acht',
 'acid',
 'acidic',
 'acker',
 'acknowledgement',


In [21]:
feature_lst10000=sw

## 2) textcuboid 생성

In [22]:
x_train=to_txt[:16000]
x_test=to_txt[16000:]

In [23]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [24]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [ ]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(16000):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

In [ ]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(4000):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

In [25]:
#1-Channel TextCuboid 생성
textcuboid=[]
if cnt>-1:      #cross vaidation을 위한 2번째 이후의 시행인 경우
    for i in train_idx:
        frame1=np.zeros((53,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid.append(frame1)  #train data의 일부 문서에 대한 TexrCuboid가 추가됨
        
    for i in range(4000):
        frame1=np.zeros((53,768))  #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in test_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid.append(frame1)  #test data의 문서에 대한 TexrCuboid가 추가됨
        

    textcuboid=np.array(textcuboid)

    np.save('./1-Channel textcuboid_WELFake(bert).npy',textcuboid)   
    
else:   #첫번째 시행인 경우
    for i in range(16000):
        frame1=np.zeros((53,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid.append(frame1)

    textcuboid=np.array(textcuboid)

    np.save('./1-Channel textcuboid_WELFake(bert).npy',textcuboid)

In [26]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

if cnt>-1:
    for i in test_idx:
        frame1=np.zeros((53,768))
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid_test.append(frame1)
        
    textcuboid_test=np.array(textcuboid_test)
    np.save('./1-Channel textcuboid_test_WELFake(bert).npy',textcuboid_test)
    
else:
    for i in range(4000):
        frame1=np.zeros((53,768))
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in test_lst[i]:
                frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid_test.append(frame1)

    textcuboid_test=np.array(textcuboid_test)
    np.save('./1-Channel textcuboid_test_WELFake(bert).npy',textcuboid_test)